In [8]:
# Data Cleaning Module
# Intakes intake and outcome data sets provided by the Austin Animal Center 
# -- Removes null values and duplicate visits
# -- Converts age strings to days (int)
# -- Merges intake and outcome datasets into one file
# -- Logic checks for age- & reproductive- related conditions 
# Outputs 2 csv files: 
# -- one of animals that have no recorded outcomes, 
# -- one that is the cleaned, merged dataset of animals with known intake and outcome data
#       This second file is suitable for all sorts of data exploration, machine learning, etc.

In [9]:
import pandas as pd
import re

In [14]:
def drop_null_data(df):
    # identifies null data in provided data frame, drops it, and reports columns in which the data were null
    # generates a Series with index = df's columns, values = number of nulls per column
    drops = df.isnull().sum()

    #if there are no nulls in the df, print & return original df
    if drops.sum() == 0:
        print("No null values identified")
            
    #if there are nulls in the df, drop them all and print which columns contained nulls and how many were dropped; returns df without nulls    
    else:
        df = df.dropna(axis = 0)
        for index, value in drops.items():
            if value != 0:
                print('"{}" contained {} null value(s)'.format(index, value))
    return df

# Regex for age parsing
year_pattern = re.compile(r"(-?\d+)\s*(?:year|years)")
month_pattern = re.compile(r"(-?\d+)\s*(?:month|months)")
week_pattern = re.compile(r"(-?\d+)\s*(?:week|weeks)")
day_pattern = re.compile(r"(-?\d+)\s*(?:day|days)")

# Function to convert age-strings to days (int)
def convert_to_days(age_str):
    if "year" in age_str:
        value = re.search(year_pattern, age_str).group(1)
        return int(value) * 365
    elif "month" in age_str:
        value = re.search(month_pattern, age_str).group(1)
        return int(value) * 30 
    elif "week" in age_str:
        value = re.search(week_pattern, age_str).group(1)
        return int(value) * 7
    elif "day" in age_str:
        value = re.search(day_pattern, age_str).group(1)
        return int(value)
    else:
        print(f"Unknown pattern: {age_str}; -1 days reported")
        return int(-1)
    
# Function to split nursing juveniles from nursing adults
def nursing_split(row):
    if row['Intake Condition'] == 'Nursing':
        if row['Intake Age'] <= 60:
            return 'Nursing Juvenile'
        elif (row['Intake Age'] > 120) & ((row['Sex upon Intake'] == 'Intact Female') | (row['Sex upon Intake'] == 'Spayed Female')):
            return 'Nursing Adult'
    return row['Intake Condition']

# Function to split sex from reproductive status
def split_sex_and_repro(row):
    sex_upon_intake = row['Sex upon Intake']
    sex_upon_outcome = row['Sex upon Outcome']
    
    # Define a dictionary mapping intake/outcome statuses to sex and reproductive status
    status_mapping = {
        'Spayed Female': {'sex': 'Female', 'repro_status': 'Altered'},
        'Intact Female': {'sex': 'Female', 'repro_status': 'Intact'},
        'Neutered Male': {'sex': 'Male', 'repro_status': 'Altered'},
        'Intact Male': {'sex': 'Male', 'repro_status': 'Intact'},
    }
    
    # Determine sex based on intake status
    sex_info = status_mapping.get(sex_upon_intake, {'sex': 'Unknown', 'repro_status': 'Unknown'})
    sex = sex_info['sex']
    repro_status_in = sex_info['repro_status']
    
    # Determine reproductive status based on outcome status, keeping sex unchanged
    repro_status_out = status_mapping.get(sex_upon_outcome, {'sex': 'Unknown', 'repro_status': 'Unknown'}).get('repro_status', 'Unknown')
        
    return sex, repro_status_in, repro_status_out

#Read files
intake_data = pd.read_csv('Austin_Animal_Center_Intakes_20240701.csv')
outcome_data = pd.read_csv('Austin_Animal_Center_Outcomes_20240701.csv')

#Convert DateTime str to numerical datetime
intake_data['DateTime'] = pd.to_datetime(intake_data['DateTime'], format = '%m/%d/%Y %I:%M:%S %p')
outcome_data['DateTime'] = pd.to_datetime(outcome_data['DateTime'], format = '%m/%d/%Y %I:%M:%S %p')

#Sorting the datasets by DateTime and dropping rows that have the same Animal ID & DateTime 
# (ie. true duplicate entries instead of animals with multiple visits)
intake_data = intake_data.sort_values(by = 'DateTime')
outcome_data = outcome_data.sort_values(by = 'DateTime')

intake_data = intake_data.drop_duplicates(subset=['Animal ID', 'DateTime'])
outcome_data = outcome_data.drop_duplicates(subset=['Animal ID', 'DateTime'])

# Merge the intake and outcome data sets based on Animal ID 
merged_data_unfiltered = pd.merge(intake_data, outcome_data, on='Animal ID', suffixes=('_intake', '_outcome'))

# Making sure the intake DateTime is earlier than the outcome DateTime to ensure pets admitted prior to data collection started have visit dates line up correctly
merged_data = merged_data_unfiltered[merged_data_unfiltered['DateTime_outcome'] >= merged_data_unfiltered['DateTime_intake']]

# Dropping columns of redundant variables or those not likely to be useful
merged_data = merged_data.drop(['Name_intake', 'MonthYear_intake', 'Found Location', 'Name_outcome', 'MonthYear_outcome', 'Animal Type_outcome', 'Breed_outcome', 'Color_outcome'], axis = 1)

# Renaming & rearranging columns to improve organization.
merged_data = merged_data.rename({'DateTime_intake': 'Intake DateTime', 'Animal Type_intake': 'Animal Type', 'Breed_intake': 'Breed', 'Color_intake':'Color', 'DateTime_outcome': 'Outcome DateTime', 'Age upon Intake': 'Intake Age', 'Age upon Outcome':'Outcome Age'}, axis='columns')
merged_data = merged_data.reindex(columns = ['Animal ID', 'Animal Type', 'Date of Birth', 'Breed', 'Color', 'Intake Type', 'Intake Condition', 'Sex upon Intake', 'Intake Age', 'Intake DateTime', 'Outcome DateTime', 'Outcome Type', 'Outcome Subtype', 'Sex upon Outcome', 'Outcome Age'])

#Fill null subtypes with "Not Specified"
merged_data['Outcome Subtype'] = merged_data['Outcome Subtype'].fillna(value = 'Not Specified')

#Flag animals w/o recorded outcomes
lost_to_follow_up = merged_data.loc[merged_data['Outcome Type'].isnull()]
print("{} animals have no outcome recorded".format(len(lost_to_follow_up)))

# Drop null values & report from whence they came
merged_data = drop_null_data(merged_data)

# Convert age strings to days (int)
merged_data['Intake Age'] = merged_data['Intake Age'].apply(convert_to_days)
merged_data['Outcome Age'] = merged_data['Outcome Age'].apply(convert_to_days)

# Dropping rows with negative ages (which also includes any age strings that were not successfully parsed)
pre = len(merged_data)
merged_data = merged_data.loc[merged_data['Intake Age'] >= 0]
print(f"Dropped rows due to negative ages: {pre - len(merged_data)}")

#Sanity check - dropping pets with "Aged" Intake Condition who are < 1yr old
pre = len(merged_data)
merged_data = merged_data[(merged_data['Intake Condition'] != 'Aged') |
                          (merged_data['Intake Age'] > 730)]
print(f"Dropped rows due to 'Aged' Intake Condition when age is under 2 years old: {pre - len(merged_data)}")

#Sanity check - dropping pets with "Neonate" Intake Condition who are > 30d old
pre = len(merged_data)
merged_data = merged_data[(merged_data['Intake Condition'] != 'Neonatal') |
                          (merged_data['Intake Age'] < 30)]
print(f"Dropped rows due to 'Neonatal' Intake Condition when age is over 30 days old: {pre - len(merged_data)}")

#Sanity check - dropping pets with "Pregnant" Intake Condition who are not Intact Females
pre = len(merged_data)
merged_data = merged_data.loc[(merged_data['Intake Condition'] != 'Pregnant') |
                (merged_data['Sex upon Intake'] == 'Intact Female')]
print(f"Dropped rows due to 'Pregnant' Intake Condition when animals is not intact female: {pre - len(merged_data)}")

# Sanity check - splitting "Nursing" Intake condition into juveniles & adults; 
# dropping the pets with "Nursing" Intake condition that are male or cannot reliably be assigned adult vs juvenile nursing status
merged_data['Intake Condition'] = merged_data.apply(nursing_split, axis = 1)
pre = len(merged_data)
merged_data = merged_data[merged_data['Intake Condition'] != 'Nursing']
print(f"Dropped rows due to 'Nursing' Intake Condition when animals is not female or when adult nursing vs nursing juvenile cannot be identified: {pre - len(merged_data)}")

# Splitting sex from reproductive status (ex. "spayed female" becomes Sex: Female, Reproductive Status: Altered)
merged_data['Sex'], merged_data['Intake Reproductive Status'], merged_data['Outcome Reproductive Status'] = zip(*merged_data.apply(split_sex_and_repro, axis=1))
merged_data = merged_data.drop(['Sex upon Intake', 'Sex upon Outcome'], axis = 1)

# Reordering columns for organization
merged_data = merged_data.reindex(columns = ['Animal ID', 'Animal Type', 'Date of Birth', 'Sex', 'Breed', 'Color', 'Intake Type', 'Intake Condition', 'Intake Age', 'Intake Reproductive Status', 'Intake DateTime', 'Outcome DateTime', 'Outcome Type', 'Outcome Subtype', 'Outcome Age', 'Outcome Reproductive Status'])

# Writes final files to csv for further review/handling
lost_to_follow_up.to_csv('Animals Without Recorded Outcomes.csv')
merged_data.to_csv('Intake and Outcome Data - Austin Animal Center.csv')

49 animals have no outcome recorded
"Sex upon Intake" contained 2 null value(s)
"Intake Age" contained 1 null value(s)
"Outcome Type" contained 49 null value(s)
"Sex upon Outcome" contained 2 null value(s)
"Outcome Age" contained 10 null value(s)
Dropped rows due to negative ages: 34
Dropped rows due to 'Aged' Intake Condition when age is under 2 years old: 21
Dropped rows due to 'Neonatal' Intake Condition when age is over 30 days old: 75
Dropped rows due to 'Pregnant' Intake Condition when animals is not intact female: 36
Dropped rows due to 'Nursing' Intake Condition when animals is not female or when adult nursing vs nursing juvenile cannot be identified: 42
